In [1]:
!pip install konlpy
from konlpy.tag import Komoran
import requests
import lxml.html
import cssselect as external_cssselect
from lxml.cssselect import CSSSelector
import tqdm
import time

You are using pip version 10.0.1, however version 19.2.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [2]:
from dateutil.relativedelta import relativedelta
from datetime import datetime, timedelta

datetime_month = datetime.now()-relativedelta(months=1)
month_ago =  datetime_month.strftime("%Y%m%d") #strftime() 은 지정된 포맷에 맞춰 date 객체의 정보를 문자열로 반환한다. month_ago는 오늘날짜의 한달 전
today = datetime.today().strftime("%Y%m%d")    #today는 오늘 날짜 반환

url = 'https://search.daum.net/search?nil_suggest=btn&w=news&DA=STC&cluster=y&q=화재원인&period=y&sd='+ month_ago +'000001&ed=' +  today +'235959'
url

'https://search.daum.net/search?nil_suggest=btn&w=news&DA=STC&cluster=y&q=화재원인&period=y&sd=20190708000001&ed=20190808235959'

In [15]:
urls = [] 
for page in range(1, ): 
    res = requests.get(url.format(page)) 
    root = lxml.html.fromstring(res.text) 
    for link in root.cssselect('a.f_nb'): 
         urls.append(link.attrib['href'])
len(urls)

12

In [16]:
articles = [] 
i=0

for u in tqdm.tqdm_notebook(urls): 
    if not u.startswith('http'): 
        continue
    try :
        res = requests.get(u) 
        time.sleep(1)
        print(i ,' ',res.status_code)
        i = i+1
        root = lxml.html.fromstring(res.text) 
        body = root.cssselect('.article_view').pop() 
        content = body.text_content().strip() # 본문을 가져와 앞뒤 공백을 제거
        articles.append(content) 
    except :
        pass


0   200
1   200
2   200
3   200
4   200
5   200
6   200
7   200



In [17]:
import pandas
df = pandas.DataFrame({'article': articles}) 
# df.to_excel('daum_news_ai.xlsx')
df.head()

,article
0,"(안성=연합뉴스) 홍기원 기자 = 소방관 2명이 사상하고, 9명의 부상자가 발생한 ..."
1,"(안성=연합뉴스) 홍기원 기자 = 소방관 2명이 사상하고, 9명의 부상자가 발생한 ..."
2,(안성=뉴스1) 조태형 기자 = 7일 오전 경기도 안성시 양성면의 공장 화재 현장에...
3,오늘(28일) 아침 7시 15분쯤 경기도 포천시에 있는 군부대 보급창고에서 불이 났...
4,"(안성=연합뉴스) 홍기원 기자 = 소방관 2명이 사상하고, 9명의 부상자가 발생한 ..."


In [18]:
from konlpy.tag import Twitter #+코모란
tagger = Twitter()

C:\Users\user\Anaconda3\lib\site-packages\konlpy\tag\_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


In [19]:
from sklearn.feature_extraction.text import CountVectorizer
def kor_noun(text): 
    words = [] 
    for w in tagger.nouns(text): 
        if len(w) > 1: 
             words.append(w) 
    return words


In [20]:
cv = CountVectorizer(tokenizer=kor_noun, max_features=1000)
articles = df['article'].astype(str)
tdm = cv.fit_transform(articles)
words = cv.get_feature_names()
#pd.DataFrame(imageKeyword_20190807).to_excel('imageKeyword_20190807.xlsx')


import pandas as pd
imageKeyword_20190807=sorted(list(zip(tdm.sum(axis=0).flat, words)), reverse=True)[:100]

In [21]:
list(imageKeyword_20190807[1])

[8, '현장']

In [23]:
keyword_20190807=list(pd.DataFrame(imageKeyword_20190807).iloc[[1,4,5,7],1]) #수작업해야 할 필요 있음
keyword_20190807

['현장', '기자', '경기도', '오전']

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import json
import os
import urllib
import argparse
import time
from datetime import datetime

#찾고자 하는 검색어를 url로 만들어 준다.
for corekeyword in keyword_20190807:
    
    searchterm = corekeyword
    url = "https://www.google.co.in/search?q="+searchterm+"&source=lnms&tbm=isch"
    # chrom webdriver 사용하여 브라우저를 가져온다.
    browser = webdriver.Chrome('C:/chromedriver')
    browser.get(url)

    # User-Agent를 통해 봇이 아닌 유저정보라는 것을 위해 사용
    header={'User-Agent':"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/43.0.2357.134 Safari/537.36"}
    # 이미지 카운트 (이미지 저장할 때 사용하기 위해서)
    counter = 0
    succounter = 0
    def get_today():
        global datetime 
        datetime = datetime.today().strftime("%Y-%m-%d") #오늘 날짜로 파일 만들기
        return datetime
    
    def make_folder(work_dir):
        if not os.path.isdir(work_dir):
            os.mkdir(work_dir)
            
    root_dir = "C:/Users/user"
    today = get_today()
    work_dir =  root_dir + "/" + today
    make_folder(work_dir)


    os.chdir(work_dir)
    # 크롤링을 하는 날짜로 만든 폴더명에 접속, 이 폴더에 '검색어' 폴더가 없으면 만들어준다.(이미지 저장 폴더를 위해서)
    if not os.path.exists(searchterm):
        os.mkdir(searchterm)

    for _ in range(200):
        browser.execute_script("window.scrollBy(0,10000)")


    for x in browser.find_elements_by_xpath('//div[contains(@class,"rg_meta")]'): # div태그에서 class name이 rg_meta인 것을 찾아온다
        counter = counter + 1
        print("Total Count:", counter)
        print("Succsessful Count:", succounter)
        print("URL:",json.loads(x.get_attribute('innerHTML'))["ou"])

        # 이미지 url
        img = json.loads(x.get_attribute('innerHTML'))["ou"]
        # 이미지 확장자
        imgtype = json.loads(x.get_attribute('innerHTML'))["ity"]

        # 구글 이미지를 읽고 저장한다.
        try:
            #req = urllib.request(img, headers={'User-Agent': header})
            raw_img = urllib.request.urlopen(img).read()
            File = open(os.path.join(searchterm , searchterm + "_" + str(counter) + "." + imgtype), "wb")
            File.write(raw_img)
            File.close()
            succounter = succounter + 1
        except:
                print("can't get img")
    os.chdir(root_dir) #다시 root_dir로 돌아오기.
    print(succounter, "pictures succesfully downloaded")

Total Count: 1
Succsessful Count: 0
URL: http://dn.joongdo.co.kr/mnt/images/file/2019y/08m/06d/2019080601000463100016721.jpg
Total Count: 2
Succsessful Count: 1
URL: http://www.hdec.kr/Common/CMM/FileDownload.aspx?attIdx=6027
can't get img
Total Count: 3
Succsessful Count: 1
URL: http://www.kyongbuk.co.kr/news/photo/201708/1002750_282132_1032.jpg
Total Count: 4
Succsessful Count: 2
URL: https://upload.wikimedia.org/wikipedia/commons/thumb/7/78/JourneytotheWest.jpg/450px-JourneytotheWest.jpg
Total Count: 5
Succsessful Count: 3
URL: http://image.chosun.com/sitedata/image/201807/04/2018070400044_0.jpg
Total Count: 6
Succsessful Count: 4
URL: http://file.mk.co.kr/meet/neds/2019/04/image_readtop_2019_202965_15542692593695682.jpg
Total Count: 7
Succsessful Count: 5
URL: https://t1.daumcdn.net/cfile/tistory/2406FA375775DBA936
Total Count: 8
Succsessful Count: 6
URL: https://t1.daumcdn.net/cfile/tistory/27632547592CDA2741
Total Count: 9
Succsessful Count: 7
URL: http://img.hani.co.kr/imgdb/res